In [5]:
import pandas as pd
from modules.data import Data
# from modules.search import Search
# from modules.video import Video
# from modules.analyze import Analyze
# from tqdm.notebook import tqdm
import csv

In [6]:
df_comments = pd.read_csv("generated_tables/comment_threads_v2.csv", index_col=False)
df_video = pd.read_csv("generated_tables/video_info.csv", index_col=False)
df_video_tagged = pd.read_csv("tagged_v3.csv", index_col=False)

In [7]:
data_obj = Data()

In [8]:
df_video_numbered = data_obj.generate_id_lookup(df=df_video, unique_column="video_url", len_identifier=5, rename_to="video_number")
df_comments["video_url"] = df_comments["video_ID"].apply(lambda video_ID: data_obj.generate_url("video", video_ID))
print("Generated video_url")
df_comments = pd.merge(left=df_comments, right=df_video_numbered, on="video_url", validate="many_to_one", suffixes=("", "_DROP")).filter(regex="^(?!.*_DROP)")
print("Merged")
df_comment_thread_numbered = data_obj.generate_id_lookup(df=df_comments, unique_column="comment_thread_ID", len_identifier=6, rename_to="comment_thread_number")
print("Generated comment_thread_number")
df_comments = pd.merge(left=df_comments, right=df_comment_thread_numbered, on="comment_thread_ID", validate="many_to_one", suffixes=("", "_DROP")).filter(regex="^(?!.*_DROP)")
print("Merged")
df_comment_numbered = data_obj.generate_id_lookup(df=df_comments, unique_column="comment_ID", len_identifier=7, rename_to="comment_number")
print("Generated comment_number")
df_comments = pd.merge(left=df_comments, right=df_comment_numbered, on="comment_ID", validate="many_to_one", suffixes=("", "_DROP")).filter(regex="^(?!.*_DROP)")
print("Merged")

# Getting a list of all channel_urls (from comments and video datasets)
df_comments["author_channel_url"] = df_comments["author_channel_ID"].apply(lambda channel_ID: data_obj.generate_url("channel", channel_ID))
list_author_channel_url = list(set(df_comments["author_channel_url"].tolist() + df_video["channel_url"].tolist()))
df_author_channel_url = pd.DataFrame(pd.Series(list_author_channel_url), columns=["author_channel_url"])
df_author_channel_url_numbered = data_obj.generate_id_lookup(df=df_author_channel_url, unique_column="author_channel_url", len_identifier=7, rename_to="author_channel_url_number")
print("Generated author_channel_url_number")
df_comments = pd.merge(left=df_comments, right=df_author_channel_url_numbered, on="author_channel_url", validate="many_to_one", suffixes=("", "_DROP")).filter(regex="^(?!.*_DROP)")
print("Merged")

Generated video_url
Merged
Generated comment_thread_number
Merged


KeyboardInterrupt: 

In [ ]:
df_comments = df_comments.drop(columns=["video_ID", "comment_thread_ID", "comment_ID", "author_channel_ID", "author_channel_url", "video_url"])

In [ ]:
df_comments.head()

,comment,like_count,updated_at,publish_time,video_number,comment_thread_number,comment_number,author_channel_url_number
0,All my weed is from the same dispensary (Truli...,0.0,2021-02-12T05:40:14Z,2021-02-12T05:40:14Z,00048,000000,0000000,0013919
1,All my weed is from the same dispensary (Truli...,0.0,2021-02-12T05:40:14Z,2021-02-12T05:40:14Z,00048,000000,0000000,0013919
2,I’m vaping rn (From FL),1.0,2021-02-08T14:57:54Z,2021-02-08T14:54:23Z,00048,000001,0000002,0013919
3,I’m vaping rn (From FL),1.0,2021-02-08T14:57:54Z,2021-02-08T14:54:23Z,00048,000001,0000002,0013919
4,Omg they are so funny and down to earth😂. New ...,0.0,2020-04-20T18:23:10Z,2020-04-20T18:23:10Z,00048,000002,0000004,0006730


In [ ]:
# Generate the anonymized video dataset
df_video = df_video.rename(columns={"channel_url": "author_channel_url"})
df_video = pd.merge(left=df_video, right=df_author_channel_url_numbered, on="author_channel_url", validate="many_to_one", suffixes=("", "_DROP")).filter(regex="^(?!.*_DROP)")
print("Anonymized author_channel_url from video dataset")
df_video = pd.merge(left=df_video, right=df_video_numbered, on="video_url", validate="one_to_one", suffixes=("", "_DROP")).filter(regex="^(?!.*_DROP)")
print("Anonymized video_url from video dataset")
# Merge with the tagged dataset
df_video = pd.merge(left=df_video, right=df_video_tagged, on="video_url", validate="many_to_one", suffixes=("", "_DROP")).filter(regex="^(?!.*_DROP)")
print("Merged with tagged video dataset")

Anonymized author_channel_url from video dataset
Anonymized video_url from video dataset
Merged with tagged video dataset


In [ ]:
# Drop video_url and channel_url
df_video = df_video.drop(columns=["video_url", "author_channel_url"])

In [ ]:
# Save the mapping
df_video_numbered.to_csv("unique_id_map/video_numbered.csv", index=False, quoting=csv.QUOTE_ALL)
df_comment_thread_numbered.to_csv("unique_id_map/comment_thread_numbered.csv", index=False, quoting=csv.QUOTE_ALL)
df_comment_numbered.to_csv("unique_id_map/comment_numbered.csv", index=False, quoting=csv.QUOTE_ALL)
df_author_channel_url_numbered.to_csv("unique_id_map/author_channel_url_numbered.csv", index=False, quoting=csv.QUOTE_ALL)

In [ ]:
# Save the anonymized comments dataset
df_comments.to_csv("unique_id_map/comments_anonymized.csv", index=False, quoting=csv.QUOTE_ALL)

In [ ]:
df_video.head()

,video_title,video_description,view_count,like_count,dislike_count,favorite_count,comment_count,publish_time,video_duration,author_channel_url_number,video_number,category,theme
0,#vAustinL Vape Trick Tutorial - How to Vape O...,"If you’re new to tricking, have no fear. It ju...",37391,565,18,0,23,2019-05-08T06:37:58Z,PT5M11S,0005292,00000,Risky,Vaping tricks
1,$100 Vape vs $5 Vape,Expensive box mod vs a regular disposable stic...,363589,4730,237,0,217,2019-09-21T17:19:33Z,PT5M25S,0023761,00001,Risky,Marketing
2,New Studio Vape Trick Session,Catch me live on twitch! https://www.twitch.tv...,2683795,64849,1075,0,1881,2019-01-11T20:16:37Z,PT3M6S,0001501,00002,Risky,Vaping tricks
3,Vape Trick Tutorial - How to: Bane French Inhale,Catch me live on twitch! https://www.twitch.tv...,4706152,68394,1511,0,1647,2017-04-14T22:58:04Z,PT3M24S,0001501,00031,Risky,Vaping tricks
4,Amazing Vape Bubble Compilation,Catch me live on twitch! https://www.twitch.tv...,17299923,242855,7676,0,7715,2018-05-26T19:42:43Z,PT3M16S,0001501,00042,Risky,Vaping tricks


In [ ]:
# Save the dataset
df_video.to_csv("unique_id_map/videos_anonymized.csv", index=False, quoting=csv.QUOTE_ALL)